In [ ]:
# Install Requirements
pip install -r requirements.txt

In [1]:
import pandas as pd
from pandas import read_csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import fbeta_score, f1_score,precision_score,recall_score,accuracy_score, roc_curve, auc
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
import copy
from credit_data_actual_values import substitute

In [2]:
# Load Data
df=read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data",sep=" ",header=None)
#df.head()
#df.info()

In [166]:
X, y = df.drop(last_ix, axis=1), df[last_ix]
print(X.shape, y.shape, Counter(y))

(1000, 20) (1000,) Counter({1: 700, 2: 300})


In [70]:
# Displaying data in human readable form with method substitute
# Make replica of the data for converting all the attribute values into human readable form
df_vis = copy.deepcopy(df)
# Call the method substitute from credit_data_actual_values.py to display the real world values
df_vis = substitute(df_vis)
df_vis.head()

,Status of existing checking account,Duration in month,Credit history,Purpose,Credit amount,Savings account/bonds,Present employment since,Installment rate in percentage of disposable income,Personal status and sex,Other debtors / guarantors,...,Property,Age in years,Other installment plans,Housing,Number of existing credits at this bank,Job,Number of people being liable to provide maintenance for,Telephone,foreign worker,Cost Matrix(Risk)
0,<0 DM,6,critical account,radio/television,1169,no savings account,>=7 years,4,male:single,none,...,real estate,67,none,own,2,skilled employee / official,1,yes,yes,Good Risk
1,0 <= <200 DM,48,existing credits paid back duly till now,radio/television,5951,<100 DM,1<= < 4 years,2,female:divorced/separated/married,none,...,real estate,22,none,own,1,skilled employee / official,1,none,yes,Bad Risk
2,no checking account,12,critical account,education,2096,<100 DM,4<= <7 years,2,male:single,none,...,real estate,49,none,own,1,unskilled - resident,2,none,yes,Good Risk
3,<0 DM,42,existing credits paid back duly till now,furniture/equipment,7882,<100 DM,4<= <7 years,2,male:single,guarantor,...,savings agreement/life insurance,45,none,for free,1,skilled employee / official,2,none,yes,Good Risk
4,<0 DM,24,delay in paying off,car (new),4870,<100 DM,1<= < 4 years,3,male:single,none,...,unknown / no property,53,none,for free,2,skilled employee / official,2,none,yes,Bad Risk


In [3]:
#Prepare Data
# split the data frame into inputs and outputs
from sklearn.preprocessing import Normalizer
Category1 = ['A11','A12','A13','A14']
Category3 = ['A30','A31','A32','A33','A34']
Category4 = ['A40','A41','A42','A43','A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A410']
Category6 = ['A61','A62','A63','A64', 'A65']
Category7 = ['A71','A72','A73','A74', 'A75']
Category9 = ['A91','A92','A93','A94', 'A95']
Category12 = ['A121','A122','A123','A124']
Category14 = ['A141','A142','A143']
Category15 = ['A151','A152','A153']
Category17 = ['A171','A172','A173', 'A174']
Category19 = ['A191','A192']
Category20 = ['A201','A202']

enc = OneHotEncoder(handle_unknown='ignore')
enc1 = OrdinalEncoder()
#enc = OneHotEncoder(categories = [Category1,Category3,Category4,Category6,Category7,Category9,Category12,Category14,Category15,Category17,Category19,Category20],handle_unknown='ignore')
lenc = LabelEncoder()
last_ix = len(df.columns) - 1
X, y = df.drop(last_ix, axis=1), df[last_ix]
#X = df.iloc[:, :20].values
#X =pd.DataFrame(X)
#print(type(X))
print(X.shape, y.shape, Counter(y))
# Categorical features has to be converted into integer values for the model to process. 
#This is done through one hot encoding.
# select categorical features
cat_ix = X.select_dtypes(include=['object']).columns
#print(cat_ix)
# one hot encode categorical features only
#ct = ColumnTransformer([('o',enc,cat_ix)], remainder='passthrough')
#print(ct)
#ct.fit(X)
enc1.fit(X)
#print("categories: ", enc.categories_)
#print("---")
#X = ct.fit_transform(X)
X = enc1.transform(df.drop(last_ix, axis=1))
#print(X[0])

# label encode the target variable to have the classes 0 and 1
#print(y)
y = lenc.fit_transform(y)
#print(y)
#print(X)
print(X.shape, y.shape, Counter(y))

(1000, 20) (1000,) Counter({1: 700, 2: 300})
(1000, 20) (1000,) Counter({0: 700, 1: 300})


In [4]:
#Splitting the data for training and testing
#X, y = df.drop(last_ix, axis=1), df[last_ix]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [242]:
# Declaring and training a classifier
# Logistic regression classifier
clf = LogisticRegression(penalty='l2',C=1.0, max_iter=10000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

In [5]:
# Decision Tree classifier
clf = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2)
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [22]:
# SVM classifier
clf = SVC(kernel='poly', degree=3, max_iter=300000)
clf.fit(X_train, y_train)

SVC(kernel='poly', max_iter=300000)

In [29]:
#K Neighbors
clf = KNeighborsClassifier(n_neighbors=12)
clf.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=12)

In [33]:
#K Neighbors - Euclidean
clf = KNeighborsClassifier(n_neighbors=7,p=2)
clf.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=7)

In [37]:
#K Neighbors - Manhattan
clf = KNeighborsClassifier(n_neighbors=7,p=1)
clf.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=7, p=1)

In [41]:
#Stochastic Gradient Descent
clf = SGDClassifier(loss='modified_huber',shuffle=True,random_state=101)
clf.fit(X_train, y_train)

SGDClassifier(loss='modified_huber', random_state=101)

In [44]:
#Random Forest
clf = RandomForestClassifier(n_estimators=70, oob_score=True, n_jobs=-1,random_state=101, max_features=None, min_samples_leaf = 30)
clf.fit(X_train, y_train)

RandomForestClassifier(max_features=None, min_samples_leaf=30, n_estimators=70,
                       n_jobs=-1, oob_score=True, random_state=101)

In [6]:
#Predicting for test data
y_pred = clf.predict(X_test)

In [49]:
print(X_test[0])

[  0.   0.   0.   1.   0.   0.   0.   0.   1.   1.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   1.   0.   0.   0.
   0.   0.   1.   0.   0.   1.   0.   0.   0.   1.   0.   0.   0.   0.
   1.   0.   1.   0.   1.   0.   0.   0.   1.   0.   1.   0.  12. 926.
   1.   2.  38.   1.   1.]


In [7]:
#Calculating results for various evaluation metric
precision = precision_score(y_test,y_pred, average='micro')
recall = recall_score(y_test,y_pred, average='micro')
accuracy = accuracy_score(y_test,y_pred)
f1 = f1_score(y_test,y_pred, average='macro')

In [8]:
print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1-score: {f1}")

Accuracy: 0.74
Recall: 0.74
Precision: 0.74
F1-score: 0.6739811912225706


In [246]:
#x = list(query_data.dict().values())
#print([X_test][0][5])
test1 = [["A11", 6, "A34", "A43", 1169, "A65", "A75", 4, "A93", "A101", 4, "A121", 67, "A143", "A152", 2, "A173", 1, "A192", "A201"]]
test1 = enc1.transform(test1)
test2 = [["A12", 48, "A32", "A43", 5951, "A61", "A73", 2, "A92", "A101", 2, "A121", 22, "A143", "A152", 1, "A173", 1, "A191", "A201"]]
test2 = enc1.transform(test2)
print(test1)
print(test1.shape)
print(test2)
print(test2.shape)
prediction1 = clf.predict(test1.reshape(1,-1))
p1 = lenc.inverse_transform(prediction1)
print(f"Model prediction1: {p1}")
prediction2 = clf.predict(test2.reshape(1,-1))
p2 = lenc.inverse_transform(prediction2)
print(f"Model prediction1: {p2}")

[[  0.   2.   4.   4. 142.   4.   4.   3.   2.   0.   3.   0.  48.   2.
    1.   1.   2.   0.   1.   0.]]
(1, 20)
[[  1.  29.   2.   4. 770.   0.   2.   1.   1.   0.   1.   0.   3.   2.
    1.   0.   2.   0.   0.   0.]]
(1, 20)
Model prediction1: [1]
Model prediction1: [2]


In [15]:
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree

from dtreeviz.trees import dtreeviz # remember to load the package
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

viz = dtreeviz(clf, X, y,
                target_name="target",
                feature_names=['Status of existing checking account',
                               'Duration',
                               'Credit history',
                               'Purpose',
                               'Credit amount',
                               'Savings account/bonds',
                               'Present employment since',
                               'Installment rate in percentage of disposable income',
                               'Personal status and sex',
                               'Other debtors / guarantors',
                               'Present residence since',
                               'Property',
                               'Age in years',
                               'Other installment plans',
                               'Housing',
                               'Number of existing credits at this bank',
                               'Job',
                               'Number of people being liable to provide maintenance for',
                               'Telephone',
                               'Foreign worker'
                              ],
                class_names=list(['1','2']))

viz
viz.save('t.svg')